In [2]:
from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
# resdir = r'C:\Users\jiahuic\Dropbox (University of Michigan)\Ford_CC\\'
resdir = r'F:\University of Michigan Dropbox\Jiahui Chen\Ford_CC\\'
# resdir = r'G:\Dropbox (University of Michigan)\Ford_CC\\'
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [3]:
gaspricedf = pd.read_csv(resdir+r'\\CostEmissionAssessment_revision\U.S._Regular_All_Formulations_Retail_Gasoline_Prices.csv',skiprows=5)
gasprice = np.mean(gaspricedf['Prices'][0:36])

In [ ]:
# qual lifetime
# EV aggregate
# ICEV kg
# EV kg
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
# Read base data
ctdf = pd.read_csv(resdir+r'\\2020_Gaz_counties_national_0728_tempreg.csv')
uqlst = np.unique(ctdf['tempreg'])

# Pre-allocate columns to avoid fragmentation
columns_to_add = [
    'lcaICEemissions', 'lcaHEemissions', 'lcaHEcosts', 'lcaICEcosts',
    'lcaICEemissions_AER', 'lcaHEemissions_AER'  # Add AER columns for ICE/HE
]
for year in ['24','30','40']:
    for scen in ['UC','CC','V2H','V2Hnoelec','V2H_cons','CC_rev','V2H_rev','V2Hnoelec_rev']:
        columns_to_add.extend([
            f'{year}{scen}_60costs', f'{year}{scen}_60emissions', f'{year}{scen}_60emissions_AER',
            f'{year}{scen}costs', f'{year}{scen}emissions', f'{year}{scen}emissions_AER'
        ])

# Initialize all columns with zeros
for col in columns_to_add:
    ctdf[col] = 0

# Format FIPS codes
ctdf['fips'] = ctdf.apply(lambda row: '0'+str(row['fips']) if len(str(row['fips']))==4 else str(row['fips']), axis=1)

# add population data
# Read population data
pop_df = pd.read_csv(resdir + r'\\CostEmissionAssessment_revision\Population_Adoption\coest2024alldata.csv', dtype={'STATE': str, 'COUNTY': str}, encoding='latin-1')

# Ensure STATE and COUNTY are zero-padded to 2 and 3 digits, respectively
pop_df['STATE'] = pop_df['STATE'].str.zfill(2)
pop_df['COUNTY'] = pop_df['COUNTY'].str.zfill(3)
pop_df['fips'] = pop_df['STATE'] + pop_df['COUNTY']

# Select only relevant columns
pop_df = pop_df[['fips', 'POPESTIMATE2024']]

# Merge population data into ctdf on 'fips'
ctdf = ctdf.merge(pop_df, on='fips', how='left')

# If any counties are missing population, fill with 0
ctdf['POPESTIMATE2024'] = ctdf['POPESTIMATE2024'].fillna(0)

# Constants
gasef = 10.64  # 2024 greet1 baseline gasoline

# Read ICEV and HEV data
icev = pd.read_csv(resdir+r'\\CostEmissionAssessment\res_trb\ICEV_1.csv')
hev = pd.read_csv(resdir+r'\\CostEmissionAssessment\res_trb\HEV.csv')

# Calculate emissions and costs for ICEV and HEV
for regi in range(432):
    mask = ctdf['tempreg'] == uqlst[regi]
    ctdf.loc[mask, '24ICEemissions'] = icev[['0'][0]][regi] 
    ctdf.loc[mask, '24HEemissions'] = hev[['0'][0]][regi]
    ctdf.loc[mask, '24ICEemissions_AER'] = icev[['0'][0]][regi] 
    ctdf.loc[mask, '24HEemissions_AER'] = hev[['0'][0]][regi] 
    ctdf.loc[mask, '30ICEemissions'] = icev[['0'][0]][regi] 
    ctdf.loc[mask, '30HEemissions'] = hev[['0'][0]][regi] 
    ctdf.loc[mask, '30ICEemissions_AER'] = icev[['0'][0]][regi] 
    ctdf.loc[mask, '30HEemissions_AER'] = hev[['0'][0]][regi] 
    ctdf.loc[mask, '40ICEemissions'] = icev[['0'][0]][regi] 
    ctdf.loc[mask, '40HEemissions'] = hev[['0'][0]][regi] 
    ctdf.loc[mask, '40ICEemissions_AER'] = icev[['0'][0]][regi] 
    ctdf.loc[mask, '40HEemissions_AER'] = hev[['0'][0]][regi] 
    ctdf.loc[mask, '24ICEcosts'] = icev[['0'][0]][regi] * gasprice / gasef
    ctdf.loc[mask, '24HEcosts'] = hev[['0'][0]][regi] * gasprice / gasef
    ctdf.loc[mask, '30ICEcosts'] = icev[['0'][0]][regi] * gasprice / gasef
    ctdf.loc[mask, '30HEcosts'] = hev[['0'][0]][regi] * gasprice / gasef
    ctdf.loc[mask, '40ICEcosts'] = icev[['0'][0]][regi] * gasprice / gasef
    ctdf.loc[mask, '40HEcosts'] = hev[['0'][0]][regi] * gasprice / gasef
    # For ICE/HE, set Emissions_AER = Emissions (no AER distinction)
    ctdf.loc[mask, '24ICEemissions_AER'] = ctdf.loc[mask, '24ICEemissions']
    ctdf.loc[mask, '24HEemissions_AER'] = ctdf.loc[mask, '24HEemissions']
    ctdf.loc[mask, '30ICEemissions_AER'] = ctdf.loc[mask, '30ICEemissions']
    ctdf.loc[mask, '30HEemissions_AER'] = ctdf.loc[mask, '30HEemissions']
    ctdf.loc[mask, '40ICEemissions_AER'] = ctdf.loc[mask, '40ICEemissions']
    ctdf.loc[mask, '40HEemissions_AER'] = ctdf.loc[mask, '40HEemissions']

# Process data for each year
for year in ['24','30','40']:
    # Read all scenario data files
    data_files = {
        'UC': (resdir+r'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_0_0_0_0_rev.csv',
               resdir+r'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_0_0_0_0_60_rev.csv'),
        'CC': (resdir+r'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_1_0_0_0.csv',
               resdir+r'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_1_0_0_0_60.csv'),
        'V2H': (resdir+r'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_2_0_0_0.csv',
                resdir+r'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_2_0_0_0_60.csv'),
        'V2Hnoelec': (resdir+'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_2_0_0_0_noelec.csv',
                      resdir+'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_2_0_0_0_noelec_60.csv'),
        'V2H_cons': (resdir+'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_2_0_0_0_cons.csv',
                     resdir+'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_2_0_0_0_cons_60.csv'),
        'CC_rev': (resdir+'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_1_0_0_0_rev.csv',
                   resdir+'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_1_0_0_0_rev.csv'),
        'V2H_rev': (resdir+'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_2_0_0_0_rev.csv',
                    resdir+'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_2_0_0_0_rev.csv'),
        'V2Hnoelec_rev': (resdir+'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_2_0_0_0_noelec_rev.csv',
                          resdir+'\\CostEmissionAssessment_revision\\res_trb\\20'+year+'_2_0_0_0_noelec_rev.csv')
    }
    
    # Process each scenario
    for scen, (file_std, file_60) in data_files.items():
        df_std = pd.read_csv(file_std)
        df_60 = pd.read_csv(file_60)
        
        # Check for NaN values
        if df_std['Cost'].isna().any() or df_std['Emissions'].isna().any():
            print(f"Warning: NaN values found in {scen} data")
        if df_60['Cost'].isna().any() or df_60['Emissions'].isna().any():
            print(f"Warning: NaN values found in {scen}_60 data")
        
        # Assign values
        # Map values based on county_num matching tempreg
        for idx, row in df_60.iterrows():
            mask = ctdf['tempreg'] == row['county_num']
            ctdf.loc[mask, f'{year}{scen}_60costs'] = row['Cost']
            ctdf.loc[mask, f'{year}{scen}_60emissions'] = row['Emissions']
            # If Emissions_AER exists in the file, use it; else, copy Emissions
            if 'Emissions_AER' in df_60.columns:
                ctdf.loc[mask, f'{year}{scen}_60emissions_AER'] = row['Emissions_AER']
            else:
                ctdf.loc[mask, f'{year}{scen}_60emissions_AER'] = row['Emissions']
        
        for idx, row in df_std.iterrows():
            mask = ctdf['tempreg'] == row['county_num']
            ctdf.loc[mask, f'{year}{scen}costs'] = row['Cost']
            ctdf.loc[mask, f'{year}{scen}emissions'] = row['Emissions']
            if 'Emissions_AER' in df_std.columns:
                ctdf.loc[mask, f'{year}{scen}emissions_AER'] = row['Emissions_AER']
            else:
                ctdf.loc[mask, f'{year}{scen}emissions_AER'] = row['Emissions']

# Calculate lifetime costs and emissions
for suf in ['','_60']:
    for scen in ['UC','CC','V2H','V2Hnoelec','V2H_cons']:
        ctdf[f'lca{scen}{suf}costs'] = 5 * (ctdf[f'24{scen}{suf}costs'] + 
                                           ctdf[f'30{scen}{suf}costs'] + 
                                           ctdf[f'40{scen}{suf}costs'])
        ctdf[f'lca{scen}{suf}emissions'] = 5 * (ctdf[f'24{scen}{suf}emissions'] + 
                                               ctdf[f'30{scen}{suf}emissions'] + 
                                               ctdf[f'40{scen}{suf}emissions'])
        ctdf[f'lca{scen}{suf}emissions_AER'] = 5 * (ctdf[f'24{scen}{suf}emissions_AER'] + 
                                                    ctdf[f'30{scen}{suf}emissions_AER'] + 
                                                    ctdf[f'40{scen}{suf}emissions_AER'])
for scen in ['ICE','HE']:
    suf = ''
    ctdf[f'lca{scen}{suf}costs'] = 5 * (ctdf[f'24{scen}{suf}costs'] + 
                                        ctdf[f'30{scen}{suf}costs'] + 
                                        ctdf[f'40{scen}{suf}costs'])
    ctdf[f'lca{scen}{suf}emissions'] = 5 * (ctdf[f'24{scen}{suf}emissions'] + 
                                            ctdf[f'30{scen}{suf}emissions'] + 
                                            ctdf[f'40{scen}{suf}emissions'])
    ctdf[f'lca{scen}{suf}emissions_AER'] = 5 * (ctdf[f'24{scen}{suf}emissions_AER'] + 
                                                ctdf[f'30{scen}{suf}emissions_AER'] + 
                                                ctdf[f'40{scen}{suf}emissions_AER'])
# Calculate differences between scenarios
for suf in ['','_60']:
    for item in ['emissions','costs','emissions_AER']:
        for scen0 in ['ICE','HE','UC','CC','V2Hnoelec','V2H','V2H_cons']:
            for scen1 in ['ICE','HE','UC','CC','V2Hnoelec','V2H','V2H_cons']:
                if scen0 != scen1:
                    # Handle special case for ICE/HE scenarios
                    current_suf = '' if scen0 in ['ICE', 'HE'] or scen1 in ['ICE', 'HE'] else suf
                    ctdf[f'lca{scen0}{scen1}{current_suf}{item}'] = (
                        ctdf[f'lca{scen0}{current_suf}{item}'] - 
                        ctdf[f'lca{scen1}{current_suf}{item}']
                    )




# Add duplicate row and update index
ctdf = pd.concat([ctdf, ctdf.iloc[2330:2331]])
ctdf['index'] = np.arange(ctdf.shape[0])
ctdf.set_index('index', inplace=True)
ctdf.loc[ctdf.index[-1], 'fips'] = '46113'

In [27]:
# Add EV sales penetration to ctdf (ignore case in matching, and allow match after removing ' Parish', ' City', or ' County')
# Use this match to fill penetration into ctdf; for elements in not_in_evsales, fill in value 3%

# Load EV sales penetration data
ev_sales_df = pd.read_excel(resdir + r'\\CostEmissionAssessment_revision\Population_Adoption\FOTW_1309_web_0.xlsx', skiprows=4)  # Update filename as needed

# Create a 'Short name' column in ev_sales_df to match ctdf
ev_sales_df['Short name'] = ev_sales_df['State'] + ev_sales_df['County Name']

# Lowercase for case-insensitive matching
ctdf_shortname_lower = ctdf['Short name'].str.lower()
ev_sales_shortname_lower = ev_sales_df['Short name'].str.lower()

# Also create versions with ' parish', ' city', and ' county' removed for fuzzy matching
def strip_parish_city_county(s):
    return s.replace(' parish', '').replace(' city', '').replace(' county', '')

ctdf_shortname_stripped = ctdf_shortname_lower.str.replace(' parish', '', regex=False).str.replace(' city', '', regex=False).str.replace(' county', '', regex=False)
ev_sales_shortname_stripped = ev_sales_shortname_lower.str.replace(' parish', '', regex=False).str.replace(' city', '', regex=False).str.replace(' county', '', regex=False)

# Prepare a Series to hold the penetration values
penetration = pd.Series(index=ctdf.index, dtype=float)

# Build a mapping from stripped/lowercase short names in ev_sales_df to their penetration values
# Assume the EV penetration column is named 'EV Penetration (%)' (update if needed)
ev_pen_col = 'Market Penetration'
ev_sales_map = dict(zip(ev_sales_shortname_lower, ev_sales_df[ev_pen_col]))
ev_sales_map_stripped = dict(zip(ev_sales_shortname_stripped, ev_sales_df[ev_pen_col]))

# For each row in ctdf, try to match and assign penetration
for idx, (sn_lower, sn_stripped) in enumerate(zip(ctdf_shortname_lower, ctdf_shortname_stripped)):
    val = None
    # Try direct match
    if sn_lower in ev_sales_map:
        val = ev_sales_map[sn_lower]
    # Try match with stripped ctdf name to ev_sales_map
    elif sn_stripped in ev_sales_map:
        val = ev_sales_map[sn_stripped]
    # Try match with ctdf name to stripped ev_sales_map
    elif sn_lower in ev_sales_map_stripped:
        val = ev_sales_map_stripped[sn_lower]
    # Try match with both stripped
    elif sn_stripped in ev_sales_map_stripped:
        val = ev_sales_map_stripped[sn_stripped]
    # If no match, will fill with 3% later
    penetration.iloc[idx] = val

# Find which ctdf rows did not get a value (i.e., not matched)
not_in_evsales_mask = penetration.isna()
penetration[not_in_evsales_mask] = 0.0338055828306915  # Fill unmatched with 3%

# Assign to ctdf
ctdf['ev_penetration_percent'] = penetration

# Optionally, print the unmatched short names for reference
not_in_evsales = ctdf.loc[not_in_evsales_mask, 'Short name']
print("Short names in ctdf not in ev_sales_df (case-insensitive, after removing ' Parish', ' City', or ' County'):")
print(not_in_evsales.tolist())


Short names in ctdf not in ev_sales_df (case-insensitive, after removing ' Parish', ' City', or ' County'):
['NMDo�a Ana County']


In [22]:
# Find strings in ev_sales_shortname_stripped that contain 'acadia' (case-insensitive)
acadia_matches = ev_sales_shortname_stripped[ev_sales_shortname_stripped.str.contains('acadia', case=False, na=False)]
print("Entries in ev_sales_shortname_stripped containing 'acadia':")
print(ev_sales_df.loc[acadia_matches.index])


Entries in ev_sales_shortname_stripped containing 'acadia':
     County Name State  Market Penetration  Unnamed: 3 Unnamed: 4  \
2703      Acadia    LA            0.004498         NaN        NaN   

           Short name  
2703  LAAcadia county  


In [29]:
ctdf.to_csv(resdir + r'\\CostEmissionAssessment_revision\Population_Adoption\ctdf_with_ev_penetration.csv', index=False)

In [16]:
len(not_in_evsales.tolist())

107

In [ ]:
112194 * 94971/10**6
# 

In [13]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import json
from urllib.request import urlopen

# 1. --- Load Geographical Data (US Counties GeoJSON) and Filter for Contiguous USA ---
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties_full = json.load(response)

# Define FIPS prefixes for non-contiguous states and territories
excluded_state_fips_prefixes = {'02', '15', '60', '66', '69', '72', '78'} # Alaska, Hawaii, American Samoa, Guam, Northern Mariana Islands, Puerto Rico, U.S. Virgin Islands

# Filter the GeoJSON features to include only contiguous USA counties
contiguous_counties_features = [
    feature for feature in counties_full['features']
    if feature['id'][:2] not in excluded_state_fips_prefixes
]

# Create a new GeoJSON object containing only the contiguous USA counties
counties = {
    "type": "FeatureCollection",
    "features": contiguous_counties_features
}



In [ ]:
# 2. --- Generate Data for Subplots ---
datasets = []
scenarios = ['UC','CC','V2Hnoelec','V2H']
item = 'costs'  # or 'emissions' depending on what you want to show
for scen in scenarios:
    data = ctdf[f'lca{scen}{item}']
    datasets.append(data)
# 3. --- Unify the Color Scale ---
global_zmin = min([np.min(ds) for ds in datasets])
global_zmax = max([np.max(ds) for ds in datasets])
display_zmin = -8000
display_zmax = 8000
print(f"Global Color Scale Range: zmin={global_zmin:.2f}, zmax={global_zmax:.2f}")
# 4. --- Create the Figure and Subplots ---
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type': 'geo'}] * 2] * 2,
    horizontal_spacing=0,
    vertical_spacing=0
)

# 5. --- Add Traces to Subplots in a Loop ---
for i, scen in enumerate(scenarios):
    row = i // 2 + 1
    col = i % 2 + 1
    
    fig.add_trace(
        go.Choropleth(
            geojson=counties,
            locations=ctdf['fips'],
            z=ctdf[f'lca{scen}{item}'],
            featureidkey="id",
            colorscale="rdbu_r",
            zmin=display_zmin,
            zmax=display_zmax,
            showscale=(i == 0),
            marker=dict(line=dict(width=0.1, color='#808080')),
            colorbar=dict(
                title=dict(
                    text='USD',
                    side="top"
                ),
                thickness=25,
                len=0.85,
                y=0.5,
                x=1.02,
                titlefont=dict(family="Times New Roman", size=24),
                tickfont=dict(family="Times New Roman", size=20),
                tickmode='array',
                tickvals=[-8000, -6000, -4000, -2000, 0, 2000, 4000, 6000, 8000],
                ticktext=['-8k', '-6k', '-4k', '-2k', '0', '2k', '4k', '6k', '8k']
            )
        ),
        row=row,
        col=col)

# Add floating text annotations for each scenario
for i, scen in enumerate(['Uncontrolled charging','Controlled charging','V2H (baseline home)', 'V2H (heat pump)']):
    row = i // 2 + 1
    col = i % 2 + 1
    
    # Calculate x and y positions based on subplot location
    x_pos = 0.25 if col == 1 else 0.75
    y_pos = 0.99 if row == 1 else 0.49
    
    fig.add_annotation(
        text=scen,
        xref="paper",
        yref="paper",
        x=x_pos,
        y=y_pos,
        xanchor='center',
        yanchor='middle',
        showarrow=False,
        font=dict(family="Times New Roman", size=24, color="black"),
    )

# 6. --- Update the Layout for All Geo Subplots and the Main Figure ---
fig.update_layout(
    height=500,
    width=900,
    margin={"r":0,"t":30,"l":0,"b":0},
    title_font=dict(family="Times New Roman", size=24),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Times New Roman", size=32, color='black'),
    showlegend=False
)

fig.update_geos(
    visible=False,
    scope="usa",
    center=dict(lat=38.5, lon=-98.5795),  # Centered on continental US
    projection_scale=1  # Slightly zoomed out
)

fig.write_image(resdir+r"CostEmissionAssessment_revision\figs_nocarb\Fig 1 lifetime costs.png", scale=2, width=900, height=500)
fig.show()

In [ ]:
# manuscript figure 2
scen0 = 'V2Hnoelec'
scen1 = 'CC'
ctdf[f'lca{scen0}{scen1}costs'] = ctdf[f'lca{scen0}costs'] - ctdf[f'lca{scen1}costs']
ctdf[f'lca{scen0}{scen1}emissions'] = ctdf[f'lca{scen0}emissions'] - ctdf[f'lca{scen1}emissions']
import matplotlib.pyplot as plt
import matplotlib
boxlst = []
matplotlib.rcParams['axes.unicode_minus'] = False
for scen in ['CCUC','V2HnoelecCC','V2HV2Hnoelec','V2HUC']:
    # for scen in ['CC']:
    year = ['lca'][0]
    item = ['costs'][0]
    boxlst.append(year+scen+item)
f,ax = plt.subplots(1,1, figsize=(6.8, 4),sharex=False, sharey=True)### figure config
a, b, c, d = 1, 2.2, 3.4, 5
# Create boxplots with equal widths
positions = [a, b, c, d]  # Modified positions to widen space between 1,2,3 and shrink space between 3 and 4
ax.boxplot([ctdf[boxlst[i]] for i in range(3)] + [ctdf[boxlst[3]]], 
           positions=positions,
           widths=0.5)  # Set equal width for all boxes

# Add vertical line to separate the groups
ax.axvline(x=4.25, color='gray', linestyle='--', alpha=0.5)  # Adjusted line position

ax.set_ylabel('Vehicle life time change\nin electricity cost (USD)')
ax.set_xticks([a, b, c, d])
ax.set_xticklabels([  # Updated tick positions to match new box positions
    'Controlled\ncharging\nminus\nUncontrolled\ncharging',
    'V2H (baseline home)\nminus\nControlled\ncharging', 
    'V2H (heat pump)\nminus\nV2H (baseline home)',
    'V2H (heat pump)\nminus\nUncontrolled\ncharging'],
    va='center')  # Set horizontal alignment to center
ax.tick_params(axis='x', which='major', pad=30)  # Increase padding between ticks and x-axis

plt.rcParams['font.size'] = 10
plt.rcParams["font.family"] = "Times New Roman"
plt.tight_layout()
plt.savefig(r'figs_nocarb\\'+'Fig2_CostChange_box.png',dpi=500)
# no carb in assessment, carb in opt

In [ ]:
# Figure 3 Emissions
# 2. --- Generate Data for Subplots ---
datasets = []
scenarios = ['ICE','HE','UC','CC','V2Hnoelec','V2H']
item = 'emissions'  # or 'emissions' depending on what you want to show
for scen in scenarios:
    data = ctdf[f'lca{scen}{item}']/1000
    datasets.append(data)
# 3. --- Unify the Color Scale ---
global_zmin = min([np.min(ds) for ds in datasets])
global_zmax = max([np.max(ds) for ds in datasets])
display_zmin = -60
display_zmax = 60
print(f"Global Color Scale Range: zmin={global_zmin:.2f}, zmax={global_zmax:.2f}")
# 4. --- Create the Figure and Subplots ---
fig = make_subplots(
    rows=2,
    cols=3,
    specs=[[{'type': 'geo'}] * 3] * 2,
    horizontal_spacing=0,
    vertical_spacing=0
)

# 5. --- Add Traces to Subplots in a Loop ---
for i, scen in enumerate(scenarios):
    row = i // 3 + 1
    col = i % 3 + 1
    
    fig.add_trace(
        go.Choropleth(
            geojson=counties,
            locations=ctdf['fips'],
            z=ctdf[f'lca{scen}{item}']/1000,
            featureidkey="id",
            colorscale="rdbu_r",
            zmin=display_zmin,
            zmax=display_zmax,
            showscale=(i == 0),
            marker=dict(line=dict(width=0.1, color='#808080')),
            colorbar=dict(
                title=dict(
                    text='t CO2e',
                    side="top"
                ),
                thickness=25,
                len=0.85,
                y=0.5,
                x=1.02,
                titlefont=dict(family="Times New Roman", size=24),
                tickfont=dict(family="Times New Roman", size=20),
                tickmode='array',
                tickvals=[-60, -45, -30, -15, 0, 15, 30, 45, 60],
                ticktext=['-60', '-45', '-30', '-15', '0', '15', '30', '45', '60']
            )
        ),
        row=row,
        col=col)

# Add floating text annotations for each scenario
for i, scen in enumerate(['ICEV', 'HEV', 'Uncontrolled charging', 'Controlled charging', 'V2H (baseline home)', 'V2H (heat pump)']):
    row = i // 3 + 1
    col = i % 3 + 1
    
    # Calculate x and y positions based on subplot location
    x_pos = 1/6 if col == 1 else (0.5 if col == 2 else 5/6)
    y_pos = .99 if row == 1 else 0.49
    print(x_pos,y_pos)
    fig.add_annotation(
        text=scen,
        xref="paper",
        yref="paper",
        x=x_pos,
        y=y_pos,
        xanchor='center',
        yanchor='middle',
        showarrow=False,
        font=dict(family="Times New Roman", size=24, color="black"),
        borderwidth=1,
        borderpad=4,
    )

# 6. --- Update the Layout for All Geo Subplots and the Main Figure ---
fig.update_layout(
    height=380,
    width=900,  # Increased width to accommodate 3 columns
    margin={"r":0,"t":30,"l":0,"b":0},
    title_font=dict(family="Times New Roman", size=24),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Times New Roman", size=32, color='black'),
    showlegend=False
)

fig.update_geos(
    visible=False,
    scope="usa",
    center=dict(lat=38.5, lon=-98.5795),  # Centered on continental US
    projection_scale=1  # Slightly zoomed out
)

fig.write_image(resdir+r"CostEmissionAssessment_revision\figs_nocarb\Fig 3 lifetime emissions.png", scale=2, width=900, height=380)
fig.show()

In [ ]:
# Figure SX Emissions AER
# 2. --- Generate Data for Subplots ---
datasets = []
scenarios = ['ICE','HE','UC','CC','V2Hnoelec','V2H']
item = 'emissions_AER'  # or 'emissions' depending on what you want to show
for scen in scenarios:
    data = ctdf[f'lca{scen}{item}']/1000
    datasets.append(data)
# 3. --- Unify the Color Scale ---
global_zmin = min([np.min(ds) for ds in datasets])
global_zmax = max([np.max(ds) for ds in datasets])
display_zmin = -60
display_zmax = 60
print(f"Global Color Scale Range: zmin={global_zmin:.2f}, zmax={global_zmax:.2f}")
# 4. --- Create the Figure and Subplots ---
fig = make_subplots(
    rows=2,
    cols=3,
    specs=[[{'type': 'geo'}] * 3] * 2,
    horizontal_spacing=0,
    vertical_spacing=0
)

# 5. --- Add Traces to Subplots in a Loop ---
for i, scen in enumerate(scenarios):
    row = i // 3 + 1
    col = i % 3 + 1
    
    fig.add_trace(
        go.Choropleth(
            geojson=counties,
            locations=ctdf['fips'],
            z=ctdf[f'lca{scen}{item}']/1000,
            featureidkey="id",
            colorscale="rdbu_r",
            zmin=display_zmin,
            zmax=display_zmax,
            showscale=(i == 0),
            marker=dict(line=dict(width=0.1, color='#808080')),
            colorbar=dict(
                title=dict(
                    text='t CO2e',
                    side="top"
                ),
                thickness=25,
                len=0.85,
                y=0.5,
                x=1.02,
                titlefont=dict(family="Times New Roman", size=24),
                tickfont=dict(family="Times New Roman", size=20),
                tickmode='array',
                tickvals=[-60, -45, -30, -15, 0, 15, 30, 45, 60],
                ticktext=['-60', '-45', '-30', '-15', '0', '15', '30', '45', '60']
            )
        ),
        row=row,
        col=col)

# Add floating text annotations for each scenario
for i, scen in enumerate(['ICEV', 'HEV', 'Uncontrolled charging', 'Controlled charging', 'V2H (baseline home)', 'V2H (heat pump)']):
    row = i // 3 + 1
    col = i % 3 + 1
    
    # Calculate x and y positions based on subplot location
    x_pos = 1/6 if col == 1 else (0.5 if col == 2 else 5/6)
    y_pos = .99 if row == 1 else 0.49
    print(x_pos,y_pos)
    fig.add_annotation(
        text=scen,
        xref="paper",
        yref="paper",
        x=x_pos,
        y=y_pos,
        xanchor='center',
        yanchor='middle',
        showarrow=False,
        font=dict(family="Times New Roman", size=24, color="black"),
        borderwidth=1,
        borderpad=4,
    )

# 6. --- Update the Layout for All Geo Subplots and the Main Figure ---
fig.update_layout(
    height=380,
    width=900,  # Increased width to accommodate 3 columns
    margin={"r":0,"t":30,"l":0,"b":0},
    title_font=dict(family="Times New Roman", size=24),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Times New Roman", size=32, color='black'),
    showlegend=False
)

fig.update_geos(
    visible=False,
    scope="usa",
    center=dict(lat=38.5, lon=-98.5795),  # Centered on continental US
    projection_scale=1  # Slightly zoomed out
)

fig.write_image(resdir+r"CostEmissionAssessment_revision\figs_nocarb\Fig SX lifetime emissions_AER.png", scale=2, width=900, height=380)
fig.show()

In [ ]:
# figure 4 emission difference 1 

# 2. --- Generate Data for Subplots ---
datasets = []
scenarios = ['UCICE','UCHE','CCICE','CCHE']
item = 'emissions'  # or 'emissions' depending on what you want to show
for scen in scenarios:
    data = ctdf[f'lca{scen}{item}']/1000
    datasets.append(data)
# 3. --- Unify the Color Scale ---
global_zmin = min([np.min(ds) for ds in datasets])
global_zmax = max([np.max(ds) for ds in datasets])
display_zmin = -50
display_zmax = 50
print(f"Global Color Scale Range: zmin={global_zmin:.2f}, zmax={global_zmax:.2f}")
# 4. --- Create the Figure and Subplots ---
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type': 'geo'}] * 2] * 2,
    horizontal_spacing=0,
    vertical_spacing=0
)

# 5. --- Add Traces to Subplots in a Loop ---
for i, scen in enumerate(scenarios):
    row = i // 2 + 1
    col = i % 2 + 1
    
    fig.add_trace(
        go.Choropleth(
            geojson=counties,
            locations=ctdf['fips'],
            z=ctdf[f'lca{scen}{item}']/1000,
            featureidkey="id",
            colorscale="rdbu_r",
            zmin=display_zmin,
            zmax=display_zmax,
            showscale=(i == 0),
            marker=dict(line=dict(width=0.1, color='#808080')),
            colorbar=dict(
                title=dict(
                    text='t CO2e',
                    side="top"
                ),
                thickness=25,
                len=0.85,
                y=0.5,
                x=1.02,
                titlefont=dict(family="Times New Roman", size=24),
                tickfont=dict(family="Times New Roman", size=20),
                tickmode='array',
                tickvals=[-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50],
                ticktext=['-50', '-40', '-30', '-20', '-10', '0', '10', '20', '30', '40', '50']
            )
        ),
        row=row,
        col=col)

# Add floating text annotations for each scenario ['UCICE','UCHE','CCICE','CCHE']
for i, scen in enumerate(['Uncontrolled charging<br>minus ICEV','Uncontrolled charging<br>minus HEV',
                          'Controlled charging<br>minus ICEV', 'Controlled charging<br>minus HEV']):
    col = i % 2 + 1
    row = i // 2 + 1
    # Calculate x and y positions based on subplot location
    x_pos = 0.25 if col == 1 else 0.75
    y_pos = 0.99 if row == 1 else 0.49
    
    fig.add_annotation(
        text=scen,
        xref="paper",
        yref="paper",
        x=x_pos,
        y=y_pos,
        xanchor='center',
        yanchor='middle',
        showarrow=False,
        font=dict(family="Times New Roman", size=24, color="black"),
    )

# 6. --- Update the Layout for All Geo Subplots and the Main Figure ---
fig.update_layout(
    height=500,
    width=900,
    margin={"r":0,"t":30,"l":0,"b":0},
    title_font=dict(family="Times New Roman", size=24),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Times New Roman", size=32, color='black'),
    showlegend=False
)

fig.update_geos(
    visible=False,
    scope="usa",
    center=dict(lat=38.5, lon=-98.5795),  # Centered on continental US
    projection_scale=1  # Slightly zoomed out
)

fig.write_image(resdir+r"CostEmissionAssessment_revision\figs_nocarb\Fig 4 emission changeICEV.png", scale=2, width=900, height=500)
fig.show()

In [ ]:
# figure 4 emission difference 1 

# 2. --- Generate Data for Subplots ---
datasets = []
scenarios = ['UCICE','UCHE','CCICE','CCHE']
item = 'emissions_AER'  # or 'emissions' depending on what you want to show
for scen in scenarios:
    data = ctdf[f'lca{scen}{item}']/1000
    datasets.append(data)
# 3. --- Unify the Color Scale ---
global_zmin = min([np.min(ds) for ds in datasets])
global_zmax = max([np.max(ds) for ds in datasets])
display_zmin = -50
display_zmax = 50
print(f"Global Color Scale Range: zmin={global_zmin:.2f}, zmax={global_zmax:.2f}")
# 4. --- Create the Figure and Subplots ---
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type': 'geo'}] * 2] * 2,
    horizontal_spacing=0,
    vertical_spacing=0
)

# 5. --- Add Traces to Subplots in a Loop ---
for i, scen in enumerate(scenarios):
    row = i // 2 + 1
    col = i % 2 + 1
    
    fig.add_trace(
        go.Choropleth(
            geojson=counties,
            locations=ctdf['fips'],
            z=ctdf[f'lca{scen}{item}']/1000,
            featureidkey="id",
            colorscale="rdbu_r",
            zmin=display_zmin,
            zmax=display_zmax,
            showscale=(i == 0),
            marker=dict(line=dict(width=0.1, color='#808080')),
            colorbar=dict(
                title=dict(
                    text='t CO2e',
                    side="top"
                ),
                thickness=25,
                len=0.85,
                y=0.5,
                x=1.02,
                titlefont=dict(family="Times New Roman", size=24),
                tickfont=dict(family="Times New Roman", size=20),
                tickmode='array',
                tickvals=[-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50],
                ticktext=['-50', '-40', '-30', '-20', '-10', '0', '10', '20', '30', '40', '50']
            )
        ),
        row=row,
        col=col)

# Add floating text annotations for each scenario ['UCICE','UCHE','CCICE','CCHE']
for i, scen in enumerate(['Uncontrolled charging<br>minus ICEV','Uncontrolled charging<br>minus HEV',
                          'Controlled charging<br>minus ICEV', 'Controlled charging<br>minus HEV']):
    col = i % 2 + 1
    row = i // 2 + 1
    # Calculate x and y positions based on subplot location
    x_pos = 0.25 if col == 1 else 0.75
    y_pos = 0.99 if row == 1 else 0.49
    
    fig.add_annotation(
        text=scen,
        xref="paper",
        yref="paper",
        x=x_pos,
        y=y_pos,
        xanchor='center',
        yanchor='middle',
        showarrow=False,
        font=dict(family="Times New Roman", size=24, color="black"),
    )

# 6. --- Update the Layout for All Geo Subplots and the Main Figure ---
fig.update_layout(
    height=500,
    width=900,
    margin={"r":0,"t":30,"l":0,"b":0},
    title_font=dict(family="Times New Roman", size=24),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Times New Roman", size=32, color='black'),
    showlegend=False
)

fig.update_geos(
    visible=False,
    scope="usa",
    center=dict(lat=38.5, lon=-98.5795),  # Centered on continental US
    projection_scale=1  # Slightly zoomed out
)

fig.write_image(resdir+r"CostEmissionAssessment_revision\figs_nocarb\Fig SX emission changeICEV AER.png", scale=2, width=900, height=500)
fig.show()

In [ ]:
# figure 4 emission difference 1 AER

# 2. --- Generate Data for Subplots ---
datasets = []
scenarios = ['UCICE','UCHE','CCICE','CCHE']
item = 'emissions_AER'  # or 'emissions' depending on what you want to show
for scen in scenarios:
    data = ctdf[f'lca{scen}{item}']/1000
    datasets.append(data)
# 3. --- Unify the Color Scale ---
global_zmin = min([np.min(ds) for ds in datasets])
global_zmax = max([np.max(ds) for ds in datasets])
display_zmin = -50
display_zmax = 50
print(f"Global Color Scale Range: zmin={global_zmin:.2f}, zmax={global_zmax:.2f}")
# 4. --- Create the Figure and Subplots ---
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type': 'geo'}] * 2] * 2,
    horizontal_spacing=0,
    vertical_spacing=0
)

# 5. --- Add Traces to Subplots in a Loop ---
for i, scen in enumerate(scenarios):
    row = i // 2 + 1
    col = i % 2 + 1
    
    fig.add_trace(
        go.Choropleth(
            geojson=counties,
            locations=ctdf['fips'],
            z=ctdf[f'lca{scen}{item}']/1000,
            featureidkey="id",
            colorscale="rdbu_r",
            zmin=display_zmin,
            zmax=display_zmax,
            showscale=(i == 0),
            marker=dict(line=dict(width=0.1, color='#808080')),
            colorbar=dict(
                title=dict(
                    text='t CO2e',
                    side="top"
                ),
                thickness=25,
                len=0.85,
                y=0.5,
                x=1.02,
                titlefont=dict(family="Times New Roman", size=24),
                tickfont=dict(family="Times New Roman", size=20),
                tickmode='array',
                tickvals=[-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50],
                ticktext=['-50', '-40', '-30', '-20', '-10', '0', '10', '20', '30', '40', '50']
            )
        ),
        row=row,
        col=col)

# Add floating text annotations for each scenario ['UCICE','UCHE','CCICE','CCHE']
for i, scen in enumerate(['Uncontrolled charging<br>minus ICEV','Uncontrolled charging<br>minus HEV',
                          'Controlled charging<br>minus ICEV', 'Controlled charging<br>minus HEV']):
    col = i % 2 + 1
    row = i // 2 + 1
    # Calculate x and y positions based on subplot location
    x_pos = 0.25 if col == 1 else 0.75
    y_pos = 0.99 if row == 1 else 0.49
    
    fig.add_annotation(
        text=scen,
        xref="paper",
        yref="paper",
        x=x_pos,
        y=y_pos,
        xanchor='center',
        yanchor='middle',
        showarrow=False,
        font=dict(family="Times New Roman", size=24, color="black"),
    )

# 6. --- Update the Layout for All Geo Subplots and the Main Figure ---
fig.update_layout(
    height=500,
    width=900,
    margin={"r":0,"t":30,"l":0,"b":0},
    title_font=dict(family="Times New Roman", size=24),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Times New Roman", size=32, color='black'),
    showlegend=False
)

fig.update_geos(
    visible=False,
    scope="usa",
    center=dict(lat=38.5, lon=-98.5795),  # Centered on continental US
    projection_scale=1  # Slightly zoomed out
)

fig.write_image(resdir+r"CostEmissionAssessment_revision\figs_nocarb\Fig SX emission changeICEV AER.png", scale=2, width=900, height=500)
fig.show()

In [ ]:
# manuscript figure 5
scen0 = 'V2Hnoelec'
scen1 = 'CC'
ctdf[f'lca{scen0}{scen1}costs'] = ctdf[f'lca{scen0}costs'] - ctdf[f'lca{scen1}costs']
ctdf[f'lca{scen0}{scen1}emissions'] = ctdf[f'lca{scen0}emissions'] - ctdf[f'lca{scen1}emissions']
import matplotlib.pyplot as plt
import matplotlib
boxlst = []
matplotlib.rcParams['axes.unicode_minus'] = False
for scen in ['CCUC','V2HnoelecCC','V2HV2Hnoelec','V2HUC']:
    # for scen in ['CC']:
    year = ['lca'][0]
    item = ['emissions'][0]
    boxlst.append(year+scen+item)
f,ax = plt.subplots(1,1, figsize=(6.8, 4),sharex=False, sharey=True)### figure config
a, b, c, d = 1, 2.2, 3.4, 5
# Create boxplots with equal widths
positions = [a, b, c, d]  # Modified positions to widen space between 1,2,3 and shrink space between 3 and 4
ax.boxplot([ctdf[boxlst[i]]/1000 for i in range(3)] + [ctdf[boxlst[3]]/1000], 
           positions=positions,
           widths=0.5)  # Set equal width for all boxes

# Add vertical line to separate the groups
ax.axvline(x=4.25, color='gray', linestyle='--', alpha=0.5)  # Adjusted line position

ax.set_ylabel('Vehicle life time change\nin GHG emissions (tCO2e)')
ax.set_xticks([a, b, c, d])
ax.set_xticklabels([  # Updated tick positions to match new box positions
    'Controlled\ncharging\nminus\nUncontrolled\ncharging',
    'V2H (baseline home)\nminus\nControlled\ncharging', 
    'V2H (heat pump)\nminus\nV2H (baseline home)',
    'V2H (heat pump)\nminus\nUncontrolled\ncharging'],
    va='center')  # Set horizontal alignment to center
ax.tick_params(axis='x', which='major', pad=30)  # Increase padding between ticks and x-axis

plt.rcParams['font.size'] = 10
plt.rcParams["font.family"] = "Times New Roman"
plt.tight_layout()
plt.savefig(r'figs_nocarb\\'+'Fig5_EmissionChange_box.png',dpi=1000)
# no carb in assessment, carb in opt

In [ ]:
# manuscript figure SX EmissionChange_box_AER
scen0 = 'V2Hnoelec'
scen1 = 'CC'
ctdf[f'lca{scen0}{scen1}costs'] = ctdf[f'lca{scen0}costs'] - ctdf[f'lca{scen1}costs']
ctdf[f'lca{scen0}{scen1}emissions'] = ctdf[f'lca{scen0}emissions'] - ctdf[f'lca{scen1}emissions']
ctdf[f'lca{scen0}{scen1}emissions_AER'] = ctdf[f'lca{scen0}emissions_AER'] - ctdf[f'lca{scen1}emissions_AER']
import matplotlib.pyplot as plt
import matplotlib
boxlst = []
matplotlib.rcParams['axes.unicode_minus'] = False
for scen in ['CCUC','V2HnoelecCC','V2HV2Hnoelec','V2HUC']:
    # for scen in ['CC']:
    year = ['lca'][0]
    item = ['emissions_AER'][0]
    boxlst.append(year+scen+item)
f,ax = plt.subplots(1,1, figsize=(6.8, 4),sharex=False, sharey=True)### figure config
a, b, c, d = 1, 2.2, 3.4, 5
# Create boxplots with equal widths
positions = [a, b, c, d]  # Modified positions to widen space between 1,2,3 and shrink space between 3 and 4
ax.boxplot([ctdf[boxlst[i]]/1000 for i in range(3)] + [ctdf[boxlst[3]]/1000], 
           positions=positions,
           widths=0.5)  # Set equal width for all boxes

# Add vertical line to separate the groups
ax.axvline(x=4.25, color='gray', linestyle='--', alpha=0.5)  # Adjusted line position

ax.set_ylabel('Vehicle life time change\nin GHG emissions (tCO2e)')
ax.set_xticks([a, b, c, d])
ax.set_xticklabels([  # Updated tick positions to match new box positions
    'Controlled\ncharging\nminus\nUncontrolled\ncharging',
    'V2H (baseline home)\nminus\nControlled\ncharging', 
    'V2H (heat pump)\nminus\nV2H (baseline home)',
    'V2H (heat pump)\nminus\nUncontrolled\ncharging'],
    va='center')  # Set horizontal alignment to center
ax.tick_params(axis='x', which='major', pad=30)  # Increase padding between ticks and x-axis

plt.rcParams['font.size'] = 10
plt.rcParams["font.family"] = "Times New Roman"
plt.tight_layout()
plt.savefig(r'figs_nocarb\\'+'Fig SX EmissionChange_box_AER.png',dpi=1000)
# no carb in assessment, carb in opt

In [ ]:
# # Check files in Result_v2h and Result_v2h_no directories
# outputdir = r'C:\Users\jiahuic\Dropbox (University of Michigan)\Ford_CC'

# # Get list of all subdirectories in Result_v2h and Result_v2h_no
# v2h_dirs = []
# v2h_no_dirs = []

# for root, dirs, files in os.walk(outputdir + r'\\Result_v2h'):
#     if dirs:  # Only add directories that have subdirectories
#         v2h_dirs.extend([os.path.join(root, d) for d in dirs])

# for root, dirs, files in os.walk(outputdir + r'\\Result_v2h_no'):
#     if dirs:  # Only add directories that have subdirectories
#         v2h_no_dirs.extend([os.path.join(root, d) for d in dirs])

# # Get list of all fnames from county 0
# county_num = 0
# fnames = os.listdir(outputdir + r'\\Energy consumption_adjusted_1_2_Y82_db\\' + str(county_num))
# fnames = [i for i in fnames if i[:6]=='0_0_0_']
# fnames = [i for i in fnames if r'.csv' in i][:15]

# # Generate expected filenames
# expected_files = [f[:-8]+'_50'+f[-4:] for f in fnames]

# # Check each directory
# missing_files = {}
# for dir_path in v2h_dirs + v2h_no_dirs:
#     # Skip directories that don't end with '_0_0' or contain specific year patterns
#     if not dir_path.endswith('_0_0') or any(year in dir_path for year in ['2024_1', '2030_1', '2040_1']):
#         continue
        
#     dir_files = set(os.listdir(dir_path))
#     missing = [f for f in expected_files if f not in dir_files]
#     if missing:
#         missing_files[dir_path] = missing

# # Print results
# if missing_files:
#     print("Missing files found:")
#     for dir_path, files in missing_files.items():
#         print(f"\nDirectory: {dir_path}")
#         print("Missing files:")
#         for f in files:
#             print(f"  - {f}")
# else:
#     print("All expected files are present in all directories")


In [ ]:
# figure 6 battery cycles
import os
import pandas as pd
# resdir = r'F:\University of Michigan Dropbox\Jiahui Chen\Ford_CC\\'
# resdir = r'G:\Dropbox (University of Michigan)\Ford_CC\\'
cyclst = [[],[[] for i in range(3)], [[] for i in range(3)]]
fnames = os.listdir(resdir+r'\\Energy consumption_adjusted_1_2_Y82_db\1\FC_opp_SUV\\')
fnames = [i for i in fnames if i[:6]=='0_0_0_']
fnames = [i for i in fnames if i not in os.listdir(resdir+r'\\Result_v2h\Results_80_Y82_2024_nocarb\0_0_0')]
fnames = [i for i in fnames if r'.csv' in i][:15]
for county_num in range(0,432):
    for fname in fnames:
        df = pd.read_csv(resdir+rf'\\Energy consumption_adjusted_1_2_Y82_db\{county_num}\\'+fname)
        cyclst[0].append(sum(df['SOC'])/82)
        for yeari in range(3):
            year = [2024,2030,2040][yeari]
            fname = fname.replace('_0_soc.csv', '_0_50.csv')
            df = pd.read_csv(resdir+rf'\\Result_v2h\Results_80_Y82_{year}_nocarb\{county_num}_0_0\\'+fname)
            cyclst[1][yeari].append(sum(df['sd'])/82)
            df = pd.read_csv(resdir+rf'\\Result_v2h\Results_80_Y82_{year}_cons_nocarb\{county_num}_0_0\\'+fname)
            cyclst[2][yeari].append(sum(df['sd'])/82)


In [ ]:
# figure 6: boxplot + 4 choropleth subplots in 3-row layout, shared colorbars

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import json
from urllib.request import urlopen

# --- Load Geographical Data (US Counties GeoJSON) and Filter for Contiguous USA ---
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties_full = json.load(response)

excluded_state_fips_prefixes = {'02', '15', '60', '66', '69', '72', '78'}
contiguous_counties_features = [
    feature for feature in counties_full['features']
    if feature['id'][:2] not in excluded_state_fips_prefixes
]
counties = {
    "type": "FeatureCollection",
    "features": contiguous_counties_features
}

# --- Prepare Data for Boxplot (top row) ---
scenlst = ['CCUC']
boxplot_pairs = [
    (cyclst[1][0],cyclst[0]),
    (cyclst[1][1],cyclst[0]),
    (cyclst[1][2],cyclst[0]),
    (cyclst[2][0],cyclst[0]),
    (cyclst[2][1],cyclst[0]),
    (cyclst[2][2],cyclst[0])
]

box_data = [cyclst[1][0],cyclst[0],cyclst[1][1],cyclst[0],cyclst[1][2],cyclst[0],
            cyclst[2][0],cyclst[0],cyclst[2][1],cyclst[0],cyclst[2][2],cyclst[0]]
box_names = ['2024','2030','2040','2024','2030','2040']

# --- Prepare Data for Choropleth Subplots (middle and bottom rows) ---
scenarios = ['V2H_consCC','V2H_consCC','V2H_consV2H','V2H_consV2H']


# --- Unify the Color Scale ---

display_zmin = [-4000,-50]
display_zmax = [4000,50]

# --- Create the Figure and Subplots ---
fig = make_subplots(
    rows=4,
    cols=6,
    column_widths=[0.2, 0.1, 0.2, 0.2, 0.15, 0.15],
    specs=[
        [{"type": "xy"}, {"type": "xy", "colspan": 5}, None, None, None, None],
        [None, None, None, None, None, None],
        [{"type": "geo", "colspan": 3}, None, None, {"type": "geo", "colspan": 3}, None, None],
        [{"type": "geo", "colspan": 3}, None, None, {"type": "geo", "colspan": 3}, None, None],
    ],
    row_heights=[0.32, 0.06, 0.31, 0.31],
    vertical_spacing=0.035,
    horizontal_spacing=0.
)

# --- Add Boxplot (row 1, col 1) ---
for i in range(0, len(box_data), 2):
    if i == 0:
        fig.add_trace(
            go.Box(
                y=box_data[i],
                boxpoints='outliers',
                marker_color='firebrick',
                line=dict(width=1.0),
                name='V2H',  # Add name for legend
                showlegend=True if i == 0 else False,  # Show legend only for first pair
                width=0.25,
                visible=True
            ),
            row=1, col=2
        )
        fig.add_trace(
            go.Box(
                y=box_data[i+1],
                boxpoints='outliers',
                marker_color='royalblue',
                line=dict(width=1.0),
                name='Driving',  # Add name for legend
                showlegend=True if i == 0 else False,  # Show legend only for first pair
                width=0.25,
                visible=True
            ),
            row=1, col=2
        )
    else:
        fig.add_trace(
            go.Box(
                y=box_data[i],
                boxpoints='outliers',
                marker_color='firebrick',
                line=dict(width=1.0),
                showlegend=True if i == 0 else False,  # Show legend only for first pair
                width=0.25,
                visible=True
            ),
            row=1, col=2
        )
        fig.add_trace(
            go.Box(
                y=box_data[i+1],
                boxpoints='outliers',
                marker_color='royalblue',
                line=dict(width=1.0),
                showlegend=True if i == 0 else False,  # Show legend only for first pair
                width=0.25,
                visible=True
            ),
            row=1, col=2
        )
# Set x-axis tick labels for boxplot, show axes and ticks
fig.update_xaxes(
    row=1, col=2,
    tickvals=[i*2+0.5 for i in range(len(box_names))],
    ticktext=box_names,
    tickangle=0,
    showline=True,
    showgrid=True,
    showticklabels=True,
    ticks='outside',
    mirror=True,
    color='black',
    linewidth=1
)
fig.update_yaxes(
    row=1, col=2,
    range=[0, 110],
    title_text='Annual Equivalent<br>Full Cycles',
    showline=True,
    showgrid=True,
    showticklabels=True,
    ticks='outside',
    mirror=True,
    color='black',
    linewidth=1
)

# --- Add Choropleth Subplots (now rows 3 and 4) ---
choropleth_titles = [
    'V2H, 5kW minus<br>controlled charging',
    'V2H, 5kW minus<br>V2H',
    'V2H, 5kW minus<br>controlled charging',
    'V2H, 5kW minus<br>V2H'
]

# Colorbar settings for shared colorbars
colorbar_settings = [
    dict(
        title=dict(text='USD', side="top"),
        thickness=15,
        len=0.4,
        y=0.3,
        x=0.46,
        titlefont=dict(family="Times New Roman", size=16),
        tickfont=dict(family="Times New Roman", size=16),
        tickmode='array',
        tickvals=[-6000, -4000, -2000, 0, 2000, 4000, 6000],
        ticktext=['-6000', '-4000', '-2000', '0', '2000', '4000', '6000']
    ),
    dict(
        title=dict(text='t CO2e', side="top"),
        thickness=15,
        len=0.4,
        y=0.3,
        x=.96,
        titlefont=dict(family="Times New Roman", size=16),
        tickfont=dict(family="Times New Roman", size=16),
        tickmode='array',
        tickvals=[-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50],
        ticktext=['-30', '-40', '-30', '-20', '-10', '0', '10', '20', '30', '40', '50']
    )
]
# Add vertical line at x=2.5
fig.add_shape(
    type="line",
    x0=5.5,
    y0=0,
    x1=5.5,
    y1=110,
    line=dict(
        color="black",
        width=1.5,
        dash="dash",
    ),
    row=1,
    col=2,
    xref="x",
    yref="paper"
)
choropleth_positions = [
    (3, 1),  # V2H_consCC
    (3, 4),  # V2H_consCC
    (4, 1),  # V2H_consV2H
    (4, 4)   # V2H_consV2H
]

for i, scen in enumerate(scenarios):
    item = ['costs','emissions'][i%2]
    datasets = [ctdf[f'lca{scen}{item}']/[1,1000][i%2] for scen in scenarios]
    global_zmin = min([np.min(ds) for ds in datasets])
    global_zmax = max([np.max(ds) for ds in datasets])
    print(f"Global Color Scale Range: zmin={global_zmin:.2f}, zmax={global_zmax:.2f}")
    # print(global_zmin,global_zmax)
    row, col = choropleth_positions[i]
    showscale = True
    colorbar = colorbar_settings[0] if col == 1 else colorbar_settings[1]
    print(f'lca{scen}{item}')
    fig.add_trace(
        go.Choropleth(
            geojson=counties,
            locations=ctdf['fips'],
            z=ctdf[f'lca{scen}{item}']/[1,1000][i%2],
            featureidkey="id",
            colorscale="rdbu_r",
            zmin=display_zmin[i%2],
            zmax=display_zmax[i%2],
            showscale=showscale,
            marker=dict(line=dict(width=0.1, color='#808080')),
            colorbar=colorbar if showscale else None
        ),
        row=row,
        col=col
    )
for i, scen in enumerate(['V2H, 5kW minus<br>controlled charging',
                          'V2H, 5kW minus<br>controlled charging',
                          'V2H, 5kW minus<br>V2H',
                          'V2H, 5kW minus<br>V2H']):
    col = i % 2 + 1
    row = i // 2 + 1
    # Calculate x and y positions based on subplot location
    x_pos = 0.28 if col == 1 else 0.78
    y_pos = 0.6116 if row == 1 else 0.2883
    
    fig.add_annotation(
        text=scen,
        xref="paper",
        yref="paper",
        x=x_pos,
        y=y_pos,
        xanchor='center',
        yanchor='middle',
        showarrow=False,
        font=dict(family="Times New Roman", size=18, color="black"),
    )
# --- Update Layout ---
fig.update_layout(
    height=700,
    width=900,
    margin={"r":3,"t":30,"l":3,"b":3},
    title_font=dict(family="Times New Roman", size=24),
    paper_bgcolor='white',
    font=dict(family="Times New Roman", size=18, color='black'),
    showlegend=True,  # Enable legend
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.71,  # Changed from 0.01 to 0.51 to move legend to right half
        bgcolor='rgba(255, 255, 255, 0.8)'  # Added semi-transparent background
    ),
    boxmode='group'
)

# --- Update geo subplots ---
for i, (row, col) in enumerate(choropleth_positions):
    geo_id = f'geo{i+1}' if (row, col) != (3,1) else 'geo'
    fig.update_geos(
        visible=False,
        scope="usa",
        center=dict(lat=38.5, lon=-98.5795),
        projection_scale=1,
        row=row, col=col
    )
# fig.write_image(r'figs_nocarb\\'+'Fig6_EFC_box.png', scale=2)

fig.show()

In [ ]:
# Figure S1 annual costs, 4 columns × 4 rows: top 3 rows are 2024, 2030, 2040 {scen}{item}, last row lca{scen}{item}
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Remove ICE and HE columns
scenarios = ['UC', 'CC', 'V2Hnoelec', 'V2H']
scenario_labels = ['Uncontrolled charging', 'Controlled charging', 'V2H (baseline home)', 'V2H (heat pump)']
years = [24, 30, 40]
item = 'costs'  # or 'emissions'

# --- Prepare all datasets for color scale ---
datasets = []
for year in years:
    for scen in scenarios:
        datasets.append(ctdf[f'{year}{scen}{item}'])
for scen in scenarios:
    datasets.append(ctdf[f'lca{scen}{item}'])

global_zmin = min([np.nanmin(ds) for ds in datasets])
global_zmax = max([np.nanmax(ds) for ds in datasets])
display_zmin = -60
display_zmax = 60
print(f"Global Color Scale Range: zmin={global_zmin:.2f}, zmax={global_zmax:.2f}")

# --- Create the Figure and Subplots (4 rows, 4 columns) ---
fig = make_subplots(
    rows=4,
    cols=4,
    specs=[[{'type': 'geo'}]*4 for _ in range(4)],
    horizontal_spacing=0,
    vertical_spacing=0
)

# --- Add Traces for 2024, 2030, 2040 for each scenario (top 3 rows) ---
for row, year in enumerate(years, start=1):
    for col, scen in enumerate(scenarios, start=1):
        trace = go.Choropleth(
            geojson=counties,
            locations=ctdf['fips'],
            z=ctdf[f'{year}{scen}{item}'],
            featureidkey="id",
            colorscale="rdbu_r",
            zmin=-700,
            zmax=700,
            showscale=(row == 1 and col == 1),  # Only show colorbar for the first subplot in top 3 rows
            marker=dict(line=dict(width=0.1, color='#808080')),
            colorbar=dict(
                title=dict(
                    text='USD',
                    side="top"
                ),
                thickness=25,
                len=0.6,
                y=1-0.375,
                x=1.02,
                titlefont=dict(family="Times New Roman", size=20),
                tickfont=dict(family="Times New Roman", size=16),
                tickmode='array',
                tickvals=[-1000, -800, -600, -400, -200, 0, 200, 400, 600, 800, 1000],
                ticktext=['-1000', '-800', '-600', '-400', '-200', '0', '200', '400', '600', '800', '1000']
            ) if (row == 1 and col == 1) else None
        )
        fig.add_trace(trace, row=row, col=col)

# --- Add Traces for lca{scen}{item} (last row), with their own colorbar (shared for last row only) ---
row = 4
for col, scen in enumerate(scenarios, start=1):
    trace = go.Choropleth(
        geojson=counties,
        locations=ctdf['fips'],
        z=ctdf[f'lca{scen}{item}'],
        featureidkey="id",
        colorscale="rdbu_r",
        zmin=-7000,
        zmax=7000,
        showscale=(col == 1),  # Only show colorbar for the first subplot in last row
        marker=dict(line=dict(width=0.1, color='#808080')),
        colorbar=dict(
            title=dict(
                text='USD',
                side="top"
            ),
            thickness=25,
            len=0.2,
            y=0.125,  # Place at bottom
            x=1.02,
            titlefont=dict(family="Times New Roman", size=20),
            tickfont=dict(family="Times New Roman", size=16),
            tickmode='array',
            tickvals=[-7000, 0, 7000],
            ticktext=['-7000', '0', '7000']
        ) if (col == 1) else None
    )
    fig.add_trace(trace, row=row, col=col)

# --- Add floating text annotations for each scenario (top of each column) ---
for col, scen_label in enumerate(scenario_labels, start=1):
    fig.add_annotation(
        text=scen_label,
        xref="paper",
        yref="paper",
        x=(col-0.5)/4,
        y=1.02,
        xanchor='center',
        yanchor='middle',
        showarrow=False,
        font=dict(family="Times New Roman", size=20, color="black"),
        borderwidth=1,
        borderpad=4,
    )

# --- Add year labels for each row (left of each row) ---
for row, year in enumerate(years, start=1):
    fig.add_annotation(
        text='20'+str(year),
        xref="paper",
        yref="paper",
        x=-0.01,
        y=1.02 - (row-0.5)/4,
        xanchor='center',
        yanchor='bottom',
        showarrow=False,
        font=dict(family="Times New Roman", size=20, color="black"),
    )
fig.add_annotation(
    text="Lifetime",
    xref="paper",
    yref="paper",
    x=-0.01,
    y=1.02 - (3.5)/4,
    xanchor='center',
    yanchor='bottom',
    showarrow=False,
    font=dict(family="Times New Roman", size=20, color="black"),
)

# --- Update the Layout for All Geo Subplots and the Main Figure ---
fig.update_layout(
    height=600,
    width=1000,
    margin={"r":0,"t":60,"l":60,"b":0},
    title_font=dict(family="Times New Roman", size=16),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Times New Roman", size=16, color='black'),
    showlegend=False
)

for i in range(4):
    for j in range(4):
        geo_id = f'geo{(i*4)+j+1}' if not (i == 0 and j == 0) else 'geo'
        fig.update_geos(
            visible=False,
            scope="usa",
            center=dict(lat=38.5, lon=-98.5795),
            projection_scale=1,
            row=i+1, col=j+1
        )

fig.write_image(resdir+r"CostEmissionAssessment_revision\figs_nocarb\Fig S1 costs.png", scale=2, width=1000, height=600)
fig.show()

In [ ]:
# Figure S2 annual emissions, 4 columns × 4 rows: top 3 rows are 2024, 2030, 2040 {scen}{item}, last row lca{scen}{item}
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Remove ICE and HE columns
scenarios = ['UC', 'CC', 'V2Hnoelec', 'V2H']
scenario_labels = ['Uncontrolled charging', 'Controlled charging', 'V2H (baseline home)', 'V2H (heat pump)']
years = [24, 30, 40]
item = 'emissions'  # or 'costs'

# --- Prepare all datasets for color scale ---
datasets = []
for year in years:
    for scen in scenarios:
        datasets.append(ctdf[f'{year}{scen}{item}'])
for scen in scenarios:
    datasets.append(ctdf[f'lca{scen}{item}'])

global_zmin = min([np.nanmin(ds) for ds in datasets])
global_zmax = max([np.nanmax(ds) for ds in datasets])
print(f"Global Color Scale Range: zmin={global_zmin:.2f}, zmax={global_zmax:.2f}")

# --- Create the Figure and Subplots (4 rows, 4 columns) ---
fig = make_subplots(
    rows=4,
    cols=4,
    specs=[[{'type': 'geo'}]*4 for _ in range(4)],
    horizontal_spacing=0,
    vertical_spacing=0
)

# --- Add Traces for 2024, 2030, 2040 for each scenario (top 3 rows) ---
for row, year in enumerate(years, start=1):
    for col, scen in enumerate(scenarios, start=1):
        trace = go.Choropleth(
            geojson=counties,
            locations=ctdf['fips'],
            z=ctdf[f'{year}{scen}{item}']/1000,
            featureidkey="id",
            colorscale="rdbu_r",
            zmin=-5,
            zmax=5,
            showscale=(row == 1 and col == 1),  # Only show colorbar for the first subplot in top 3 rows
            marker=dict(line=dict(width=0.1, color='#808080')),
            colorbar=dict(
                title=dict(
                    text='tCO2e',
                    side="top"
                ),
                thickness=25,
                len=0.6,
                y=1-0.375,
                x=1.02,
                titlefont=dict(family="Times New Roman", size=20),
                tickfont=dict(family="Times New Roman", size=16),
                tickmode='array',
                tickvals=[-5, -2.5, 0, 2.5, 5],
                ticktext=['-5', '-2.5', '0', '2.5', '5']
            ) if (row == 1 and col == 1) else None
        )
        fig.add_trace(trace, row=row, col=col)

# --- Add Traces for lca{scen}{item} (last row), with their own colorbar (shared for last row only) ---
row = 4
for col, scen in enumerate(scenarios, start=1):
    trace = go.Choropleth(
        geojson=counties,
        locations=ctdf['fips'],
        z=ctdf[f'lca{scen}{item}']/1000,
        featureidkey="id",
        colorscale="rdbu_r",
        zmin=-60,
        zmax=60,
        showscale=(col == 1),  # Only show colorbar for the first subplot in last row
        marker=dict(line=dict(width=0.1, color='#808080')),
        colorbar=dict(
            title=dict(
                text='tCO2e',
                side="top"
            ),
            thickness=25,
            len=0.2,
            y=0.125,  # Place at bottom
            x=1.02,
            titlefont=dict(family="Times New Roman", size=20),
            tickfont=dict(family="Times New Roman", size=16),
            tickmode='array',
            tickvals=[-60, 0, 60],
            ticktext=['-60', '0', '60']
        ) if (col == 1) else None
    )
    fig.add_trace(trace, row=row, col=col)

# --- Add floating text annotations for each scenario (top of each column) ---
for col, scen_label in enumerate(scenario_labels, start=1):
    fig.add_annotation(
        text=scen_label,
        xref="paper",
        yref="paper",
        x=(col-0.5)/4,
        y=1.02,
        xanchor='center',
        yanchor='middle',
        showarrow=False,
        font=dict(family="Times New Roman", size=20, color="black"),
        borderwidth=1,
        borderpad=4,
    )

# --- Add year labels for each row (left of each row) ---
for row, year in enumerate(years, start=1):
    fig.add_annotation(
        text='20'+str(year),
        xref="paper",
        yref="paper",
        x=-0.01,
        y=1.02 - (row-0.5)/4,
        xanchor='center',
        yanchor='bottom',
        showarrow=False,
        font=dict(family="Times New Roman", size=20, color="black"),
    )
fig.add_annotation(
    text="Lifetime",
    xref="paper",
    yref="paper",
    x=-0.01,
    y=1.02 - (3.5)/4,
    xanchor='center',
    yanchor='bottom',
    showarrow=False,
    font=dict(family="Times New Roman", size=20, color="black"),
)

# --- Update the Layout for All Geo Subplots and the Main Figure ---
fig.update_layout(
    height=600,
    width=1000,
    margin={"r":0,"t":60,"l":60,"b":0},
    title_font=dict(family="Times New Roman", size=16),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Times New Roman", size=16, color='black'),
    showlegend=False
)

for i in range(4):
    for j in range(4):
        geo_id = f'geo{(i*4)+j+1}' if not (i == 0 and j == 0) else 'geo'
        fig.update_geos(
            visible=False,
            scope="usa",
            center=dict(lat=38.5, lon=-98.5795),
            projection_scale=1,
            row=i+1, col=j+1
        )

fig.write_image(resdir+r"CostEmissionAssessment_revision\figs_nocarb\Fig S2 emissions.png", scale=2, width=1000, height=600)
fig.show()

In [ ]:
86400/3600

In [ ]:
630712800/3600/24/365

In [ ]:
# fig S3 degradation

In [ ]:
# fig S4 with carb sensitivity

In [ ]:
for scen in ['CCUC','V2HnoelecUC','V2HUC']:
    values = ctdf[f'lca{scen}costs'].dropna().sort_values().values
    n = len(values)
    # Find the closest observation to the 5th and 95th percentiles
    p5_idx = int(round(0.05 * (n - 1)))
    p95_idx = int(round(0.95 * (n - 1)))
    mean_val = np.mean(values)
    print("5th:", values[p5_idx])
    print("Mean:", mean_val)
    print("95th:", values[p95_idx])

In [ ]:
for scen in ['CCUC','V2HnoelecUC','V2HUC']:
    values = ctdf[f'lca{scen}emissions'].dropna().sort_values().values
    n = len(values)
    # Find the closest observation to the 5th and 95th percentiles
    p5_idx = int(round(0.05 * (n - 1)))
    p95_idx = int(round(0.95 * (n - 1)))
    mean_val = np.mean(values)
    print("5th:", round(values[p5_idx]/1000, 3))
    print("Mean:", round(mean_val/1000, 3))
    print("95th:", round(values[p95_idx]/1000, 3))

In [ ]:
0.93*0.62+0.035
0.93*0.31

In [ ]:
choropleth_titles